In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import logging 
import os

# Path to the log file
log_file_path = 'spam.log'

# Remove the log file if it exists
if os.path.exists(log_file_path):
    os.remove(log_file_path)

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
# create file handler which logs even debug messages
fh = logging.FileHandler(log_file_path, mode='w')
# Create a formatter and set it for the handler
formatter = logging.Formatter('%(name)s|%(levelname)s| %(message)s')
fh.setFormatter(formatter)
fh.setLevel(logging.DEBUG)
logger.addHandler(fh)

import json
import polars as pl
from polars import col, lit
import altair as alt
import simrs
from history import History, tabular
import input_builder as b
import utils 

In [3]:
# Agents
agent_ids = ["a" +str(i) for i in range(1, 10)] 

# Goods
wheat = "Wheat"
goods = [wheat]

# Ports
genoa = "Genoa"
rome = "Rome"
milan = "Milan"
venice = "Venice"
marsailles = "Marsailles"
port_ids = [genoa, rome, milan, venice, marsailles]
# port_ids = [genoa, rome]

# genoa -> milan -> marsailles
#   v         v
# rome -> venice
edges = [(genoa, milan), (milan, marsailles), (rome, venice), (rome, genoa), (venice, milan)]
# edges = [(genoa, rome)]
# _market = lambda net: b._market(wheat, b._market_info(net=net)) 
_market = lambda x: b._market(wheat, b._market_info(production=10, consumption=10-x, supply=1000+x)) 

# net balanced
ports = [
  b._port(genoa, _market(2)), 
  b._port(milan, _market(1)), 
  b._port(rome, _market(0)), 
  b._port(venice, _market(-1)),
  b._port(marsailles, _market(-2)), 
]

_agent = lambda id, pos: b._agent(id, pos, 1000, "Exhaustive")
agents = [_agent(id, port_id) for (id, port_id) in zip(agent_ids, port_ids * 20)] 

x = b._inputFormat( agents = agents, ports=ports, edges= edges, opts=b._opts(ticks=50))
with open("../../input/last.json", 'w') as fp:
  json.dump(x, fp, indent=2)
history = simrs.run(x)
(actions, agents, markets, events) = tabular(history)
markets = markets.select(pl.exclude("pricer"))

In [4]:
print("agents") 
agents

agents


behavior,cargo,coins,id,pos,tick
str,str,f64,str,str,i64
"""Exhaustive""",null,1000.0,"""a8""","""Milan""",0
"""Exhaustive""",null,1000.0,"""a2""","""Rome""",0
"""Exhaustive""",null,1000.0,"""a7""","""Rome""",0
"""Exhaustive""",null,1000.0,"""a1""","""Genoa""",0
"""Exhaustive""",null,1000.0,"""a4""","""Venice""",0
"""Exhaustive""",null,1000.0,"""a9""","""Venice""",0
"""Exhaustive""",null,1000.0,"""a6""","""Genoa""",0
"""Exhaustive""",null,1000.0,"""a3""","""Milan""",0
"""Exhaustive""",null,1000.0,"""a5""","""Marsailles""",0


In [5]:
print("markets") 
markets

markets


consumption,good,port,price,production,supply,tick
f64,str,str,f64,f64,f64,i64
11.0,"""Wheat""","""Venice""",100.2,10.0,999.0,0
9.0,"""Wheat""","""Milan""",99.8,10.0,1001.0,0
12.0,"""Wheat""","""Marsailles""",100.4,10.0,998.0,0
8.0,"""Wheat""","""Genoa""",99.6,10.0,1002.0,0
10.0,"""Wheat""","""Rome""",100.0,10.0,1000.0,0
11.0,"""Wheat""","""Venice""",100.4,10.0,998.0,1
12.0,"""Wheat""","""Marsailles""",100.8,10.0,996.0,1
10.0,"""Wheat""","""Rome""",100.4,10.0,998.0,1
9.0,"""Wheat""","""Milan""",100.0,10.0,1000.0,1


In [6]:
print("actions")
actions.head(10)

actions


action,agent_id,good,port_id,tick
str,str,str,str,i64
"""BuyAndMove""","""a8""","""Wheat""","""Venice""",0
"""BuyAndMove""","""a2""","""Wheat""","""Genoa""",0
"""BuyAndMove""","""a7""","""Wheat""","""Genoa""",0
"""BuyAndMove""","""a1""","""Wheat""","""Rome""",0
"""Move""","""a4""",null,"""Milan""",0
"""Move""","""a9""",null,"""Milan""",0
"""BuyAndMove""","""a6""","""Wheat""","""Rome""",0
"""BuyAndMove""","""a3""","""Wheat""","""Venice""",0
"""Move""","""a5""",null,"""Milan""",0


In [7]:
events.head(10)

agent,amt,cost,event,good,port,tick
str,i64,f64,str,str,str,i64
"""a8""",1,99.9,"""Trade""","""Wheat""","""Milan""",0
"""a2""",1,100.1,"""Trade""","""Wheat""","""Rome""",0
"""a7""",1,100.3,"""Trade""","""Wheat""","""Rome""",0
"""a1""",1,99.7,"""Trade""","""Wheat""","""Genoa""",0
"""a6""",1,99.9,"""Trade""","""Wheat""","""Genoa""",0
"""a3""",1,100.1,"""Trade""","""Wheat""","""Milan""",0
"""a8""",-1,-100.3,"""Trade""","""Wheat""","""Venice""",1
"""a2""",-1,-99.5,"""Trade""","""Wheat""","""Genoa""",1
"""a7""",-1,-99.3,"""Trade""","""Wheat""","""Genoa""",1


In [8]:
def plot_agents(agents: pl.DataFrame):
    base = alt.Chart(agents).encode(
        x='tick:Q',
        y=alt.Y('coins:Q').scale(zero=False),
        color=alt.Color('id:O').scale(scheme='dark2'),
    )
    lines = base.transform_loess('tick', 'coins', bandwidth=.5, groupby=['id']).mark_line(size=4)
    return (base.mark_point() + lines).interactive() 
  
plot_agents(agents)

alt.LayerChart(...)

In [9]:
def plot_agent_locations(agents: pl.DataFrame):
    base = alt.Chart(agents).encode(
        x='tick:Q',
        y='pos:N',
        color=alt.Color('id:O').scale(scheme='dark2'),
    )
    return base.mark_point().interactive()
plot_agent_locations(agents)

alt.Chart(...)

In [10]:
def plot_prices_by_port(ports: pl.DataFrame, color='dark2'):
    base = alt.Chart(ports).encode(
        x='tick:Q',
        y='price:Q',
        color=alt.Color('port:O').scale(scheme=color),
    )
    lines = base.transform_loess('tick', 'price', bandwidth=.5, groupby=['port']).mark_line(size=4)
    # return (base.mark_point() + lines).interactive() 
    return lines.interactive()
plot_prices_by_port(markets)

alt.Chart(...)

In [11]:
def make_routes(events):
    trade_events = events.filter(events["event"] == "Trade")
    def foo(df):
        buys = df.filter(df["amt"] > 0).select(
            'agent', 
            'amt', 
            pl.col("cost").alias('buy_cost'), 
            pl.col('port').alias('src'), 
            pl.col('tick').alias('buy_tick')
        )
        sells = df.filter(df["amt"] < 0).select(
            pl.col('cost').alias('sell_cost'), 
            pl.col('port').alias('dst'),
            pl.col('tick').alias('sell_tick')
        )

        df = pl.concat( [ buys, sells ], how='horizontal')
        df = df.with_columns((-df["sell_cost"] - df["buy_cost"]).alias("profit"))
        df = df.with_columns((df['buy_cost'] / df['amt']).alias('buy_price'))
        df = df.with_columns((df['sell_cost'] / -df['amt']).alias('sell_price'))
        return df

    return trade_events.groupby('agent').apply(foo)
routes = make_routes(events)
routes.head()

agent,amt,buy_cost,src,buy_tick,sell_cost,dst,sell_tick,profit,buy_price,sell_price
str,i64,f64,str,i64,f64,str,i64,f64,f64,f64
"""a3""",1,100.1,"""Milan""",0,-100.1,"""Venice""",1,0.0,100.1,100.1
"""a3""",1,100.1,"""Milan""",3,-101.1,"""Marsailles""",4,1.0,100.1,101.1
"""a3""",1,97.9,"""Genoa""",7,-100.9,"""Venice""",9,3.0,97.9,100.9
"""a3""",1,99.7,"""Milan""",11,-103.1,"""Marsailles""",12,3.4,99.7,103.1
"""a3""",1,100.1,"""Milan""",14,-102.5,"""Marsailles""",15,2.4,100.1,102.5


In [17]:
def plot_trades(trades: pl.DataFrame):
    base = alt.Chart(trades).encode(
        x='sell_tick:Q',
        y='profit:Q',
        color=alt.Color('agent:N').scale(scheme='dark2'),
    )
    return base.mark_point().interactive()
plot_trades(routes)

alt.Chart(...)

In [13]:
def plot_buy_and_sell_prices(trades: pl.DataFrame):
    buy = alt.Chart(trades).encode(
        x='buy_tick:Q',
        y='buy_price:Q',
        color=alt.Color('dst:N').scale(scheme='dark2'),
    )
    sell = alt.Chart(trades).encode(
        x='sell_tick:Q',
        y='sell_price:Q',
        color=alt.Color('dst:N').scale(scheme='dark2'),
    )
    
    return (buy.mark_point()+sell.mark_point()).interactive()

plot_buy_and_sell_prices(routes) + plot_trades(routes)

alt.LayerChart(...)

In [14]:
## Where would prices have been if agents didn't trade?
# x = b._inputFormat( agents = agents, ports=ports, edges= edges, opts=b._opts(ticks=100))
def no_agent_markets(input_format) -> pl.DataFrame:
    input_format['agents'] = []
    no_agent_history = simrs.run(input_format)
    (_, _, no_agent_markets,_) = tabular(no_agent_history)
    return no_agent_markets.select(pl.exclude("pricer"))

plot_prices_by_port(no_agent_markets(x)) 

alt.Chart(...)

In [15]:
plot_prices_by_port(markets)

alt.Chart(...)

## Port level analysis
- How much does each port trade?
- Trade volume bucketted 
- Volume in dollars 
- Biggest trading partners (other ports)

In [16]:
## Total goods traded per port
events.groupby("port").agg(pl.sum("amt"))
# def plot_

port,amt
str,i64
"""Venice""",-38
"""Genoa""",56
"""Rome""",-2
"""Marsailles""",-62
"""Milan""",52


Metrics for individual agent
- total coins
- coins per tick
- died?

Metrics for population of agents
- min, max, median, mean, std of agent coins

How well did the agents equalize prices?
- box plot of prices
- stddev of pricesf

Construct 'trades'
- bought Cargo at StartPort for BuyPrice
- sold Cargo at EndPort for SellPrice
- profit = SellPrice - BuyPrice
- Route = StartPort -> .. ->  EndPort
- RouteLength = len(Route)